In [2]:
import numpy as np

from operator import add
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.ml.feature import RegexTokenizer, CountVectorizer
from pyspark.ml.feature import StopWordsRemover, VectorAssembler
from pyspark.ml.feature import IDF
from pyspark.ml import Pipeline, PipelineModel

from pyspark.sql.functions import *
from pyspark.sql.types import *


sc = SparkSession.builder.getOrCreate()
spark = SparkSession.builder.appName('FinalProject').getOrCreate()

# Get sparkcontext from 
sqlContext = SQLContext(sc)N

df_business = spark.read.json('/Volumes/Nischay HDD/yelp_academic_dataset_business.json')

/Users/nischay/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
22/03/31 23:31:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
df_business_MA = df_business.select('business_id', 'name',  'address', 'city', \
                                      'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'categories') \
                                .filter( (df_business.city == 'Vancouver') & (df_business.categories.contains('Restaurants') ) )

business_count = df_business_MA.count()
print(f'MA Business count: {business_count}')

# Sampling data to run on local
sampled_business_MA = df_business_MA.sample(0.05)

MA Business count: 5026


In [79]:
df_business.printSchema()

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- GoodForDancing: str

In [80]:
df_business.select('business_id', 'name',  'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'categories').write.format('jdbc').options(
      url='jdbc:mysql://localhost/food_hunter_development',
      driver='com.mysql.jdbc.Driver',
      dbtable='restaurants',
      user='root',
      password='Nanu1996').mode('append').save()

22/04/03 03:19:09 ERROR Executor: Exception in task 7.0 in stage 9423.0 (TID 23119)
java.io.FileNotFoundException: 
File file:/Volumes/Nischay HDD/yelp_academic_dataset_business.json does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:187)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:104)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.catalyst.exp

Py4JJavaError: An error occurred while calling o19843.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 9423.0 failed 1 times, most recent failure: Lost task 5.0 in stage 9423.0 (TID 23117) (192.168.2.22 executor driver): java.io.FileNotFoundException: 
File file:/Volumes/Nischay HDD/yelp_academic_dataset_business.json does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:187)
	at org.apache.spark.sq

In [1]:
spark.stop()

In [6]:
sampled_business_MA.count()

2483

In [9]:
# SCRIPT YOU NEED 


# ERADING ALL JSON FROM LOCAL

df_business = spark.read.json('/Volumes/Nischay HDD/yelp_academic_dataset_business.json')
df_reviews  = spark.read.json('/Volumes/Nischay HDD/yelp_academic_dataset_review.json')
df_users  = spark.read.json('/Volumes/Nischay HDD/yelp_academic_dataset_user.json')


# FILTERING DATASET TO VANCOUVER

df_business_MA = df_business.select('business_id', 'name',  'address', 'city', \
                                      'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'categories') \
                                .filter( (df_business.city == 'Vancouver') & (df_business.categories.contains('Restaurants') ) )

business_count = df_business_MA.count()
print(f'MA Business count: {business_count}')

# Sampling data to run on local
sampled_business_MA = df_business_MA.sample(0.01)


# FETCHING USER REVIEWS FOR FILTERED BUSINESES

df_reviews_MA = df_reviews.join(sampled_business_MA, on = 'business_id', how = 'inner') \
                          .select(df_reviews.business_id, df_reviews.user_id, df_reviews.review_id, df_reviews.stars)
df_reviews_count = df_reviews_MA.count()
print(f'Business Reviews count: {df_reviews_count}')



# FETCHING USERS
df_users_MA = df_users.join(df_reviews_MA, on = 'user_id', how = 'inner') \
                          .select(df_users.user_id, df_users.name, df_users.review_count, df_users.yelping_since, \
                                  df_users.useful, df_users.funny , df_users.cool , df_users.fans , df_users.average_stars)
df_users_MA.show(2)

df_users_MA_count = df_users_MA.count()
print(f'Users count: {df_users_MA_count}')




# WRITRING THE DFs into paraquet

sampled_business_MA.coalesce(1).write.parquet('New_Small_Datasets/small_business_datset')
df_reviews_MA.coalesce(1).write.parquet('New_Small_Datasets/small_reviews_dataset')
df_users_MA.coalesce(1).write.parquet('New_Small_Datasets/small_users_dataset')







MA Business count: 5026


Business Reviews count: 16472


+--------------------+----+------------+-------------------+------+-----+----+----+-------------+
|             user_id|name|review_count|      yelping_since|useful|funny|cool|fans|average_stars|
+--------------------+----+------------+-------------------+------+-----+----+----+-------------+
|hCAuMs7R7FFh4gbkM...|Chon|         370|2009-12-25 23:15:20|   319|  127| 154|  14|         3.77|
|OyLO6fl4st6r_YX-L...|Mish|          59|2009-06-23 22:52:52|    47|   18|  35|   2|          3.8|
+--------------------+----+------------+-------------------+------+-----+----+----+-------------+
only showing top 2 rows



Users count: 16472


In [20]:
# Read data generated from above process
business_df = spark.read.parquet('New_Small_Datasets/small_business_datset')
business_count = business_df.count()

reviews_df = spark.read.parquet('New_Small_Datasets/small_reviews_dataset')
reviews_count = reviews_df.count()

users_df = spark.read.parquet('New_Small_Datasets/small_users_dataset')
users_count = users_df.count()

print(f'Business COunt: {business_count}')
print(f'Reviews COunt: {reviews_count}')
print(f'Users COunt: {users_count}')

Business COunt: 256
Reviews COunt: 16472
Users COunt: 16472


In [81]:
business_df.select('business_id', 'name',  'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'categories').write.format('jdbc').options(
      url='jdbc:mysql://localhost/food_hunter_development',
      driver='com.mysql.jdbc.Driver',
      dbtable='restaurants',
      user='root',
      password='Nanu1996').mode('append').save()

In [66]:
business_df.show(5)
reviews_df.show(5)
users_df.select("user_id").show(1)

+--------------------+--------------------+--------------------+---------+-----+-----------+-------------+--------------+-----+------------+--------------------+
|         business_id|                name|             address|     city|state|postal_code|     latitude|     longitude|stars|review_count|          categories|
+--------------------+--------------------+--------------------+---------+-----+-----------+-------------+--------------+-----+------------+--------------------+
|7ee6XyKyeduI-Rbc6...|   NingTu Restaurant|       2130 Kingsway|Vancouver|   BC|    V5N 2T5|   49.2440788|  -123.0633372|  4.0|          60|Restaurants, Chinese|
|AWat-D7msT_Yfl46P...|  The Vallarta Grill|    102 Water Street|Vancouver|   BC|    V6B 1B2|   49.2837849|  -123.1066087|  2.0|          18|Mexican, Restaurants|
|meNKSbzydv2C5RquO...|           Shirakawa|12 Water Street, ...|Vancouver|   BC|    V6B 1A5|   49.2835046|  -123.1047979|  3.5|          41|Japanese, Restaur...|
|ZkwdIVRaUpsxG6vNq...|    Fr

In [21]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession ,Row
from pyspark.sql.functions import col
from pyspark.sql import SQLContext
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import StructType,StructField,IntegerType
spark = SparkSession.builder.appName("Food Hunters").getOrCreate()

In [22]:
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in ['business_id', 'user_id']]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(reviews_df).transform(reviews_df)
reviews_df = transformed.select(['business_id', 'business_id_index', 'user_id', 'user_id_index', 'stars'])
reviews_df.show()

+--------------------+-----------------+--------------------+-------------+-----+
|         business_id|business_id_index|             user_id|user_id_index|stars|
+--------------------+-----------------+--------------------+-------------+-----+
|7ee6XyKyeduI-Rbc6...|             80.0|ObrM-7SNQ0EP10stt...|       5797.0|  1.0|
|4I8aLMG3VcmRS4etj...|             43.0|CH2eTijaxiD9YZu6g...|       1131.0|  1.0|
|dvCv7QvAmLq1PSNJb...|            234.0|XUBnDTrfeVxdxnlNa...|       7102.0|  5.0|
|s6REZ2GzuMJbPmQrl...|            163.0|QBMf37MERSJLgoPYY...|       6051.0|  1.0|
|AWat-D7msT_Yfl46P...|            167.0|6aXPS6bk5Wxf82cw1...|        340.0|  3.0|
|5nP-iJTCbDpiKjKXS...|             64.0|xYaWWlzH8XHzoizAp...|      11072.0|  5.0|
|2WRCcQATOe_Em0k61...|             66.0|ng0dtz--8RidYO1cD...|        829.0|  4.0|
|7ee6XyKyeduI-Rbc6...|             80.0|x4dMlJWPH-iaNqdDo...|      10998.0|  5.0|
|upNffrDjq_VNFd7pk...|            166.0|LQkUvIUcNCxl0BPnf...|        180.0|  3.0|
|JHdsuTOvlD1Oc5h

In [24]:
# Count the total number of stars in the dataset
numerator = reviews_df.select("stars").count()

# Count the number of distinct user_id and distinct business_id
num_users = reviews_df.select("user_id_index").distinct().count()
num_restaurants = reviews_df.select("business_id_index").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of num_restaurants
denominator = num_users * num_restaurants

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  99.44% empty.


### Creating ALS Model

In [15]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator


In [25]:
(train, test) = reviews_df.randomSplit([0.8, 0.2], seed = 1234)
als = ALS(userCol="user_id_index", itemCol="business_id_index", ratingCol="stars", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

In [26]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \

           
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  16


In [27]:
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)

CrossValidator_9b10643bcf81


In [28]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel

22/04/03 01:01:58 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/04/03 01:01:58 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


In [29]:
# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
  Rank: 150
  MaxIter: 10
  RegParam: 0.01


In [30]:
# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

1.421908999659508


In [31]:
test_predictions.show()

+--------------------+-----------------+--------------------+-------------+-----+----------+
|         business_id|business_id_index|             user_id|user_id_index|stars|prediction|
+--------------------+-----------------+--------------------+-------------+-----+----------+
|KxCxaDrBYCbLWyl3c...|             27.0|ZmZ1sgKqq8L7Kkeu2...|       1580.0|  4.0|  2.441624|
|T4QPN9UU5Z1UWmo-3...|            102.0|l5DiYlolhnlLhVCYW...|       1829.0|  5.0| 3.9755504|
|XZg7TWr4eBiVXeSca...|              5.0|rp8BECKOXsyWrwDvk...|        471.0|  2.0| 2.3914065|
|dzGssH8MPm6Z8oM8A...|              4.0|HkEvg9tJgjm28D6N0...|       1238.0|  3.0| 2.9990695|
|dzGssH8MPm6Z8oM8A...|              4.0|q-v8elVPvKz0KvK69...|        463.0|  2.0|  2.002916|
|htvl5L_V-zKN0UvUg...|              1.0|p0yILBCYCSuFL_af3...|        833.0|  4.0| 1.8942016|
|vJ-NDALMmnEB_KJSb...|             18.0|p0yILBCYCSuFL_af3...|        833.0|  4.0| 2.7564726|
|HTJTTobu5hXM2Xgqj...|              6.0|MHv9FRUZSZzqxJNUh...|        3

In [32]:
# Generate n Recommendations for all users
nrecommendations = best_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

/Users/nischay/spark/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-------------+--------------------+
|user_id_index|     recommendations|
+-------------+--------------------+
|           26|[{51, 4.9822407},...|
|           27|[{142, 4.3349915}...|
|           28|[{39, 4.981995}, ...|
|           31|[{108, 5.348727},...|
|           34|[{142, 4.7420287}...|
|           44|[{140, 4.562914},...|
|           53|[{142, 4.0742817}...|
|           65|[{117, 4.2093654}...|
|           76|[{222, 4.9870977}...|
|           78|[{11, 4.9884815},...|
+-------------+--------------------+



In [35]:
nrecommendations.join(df_business, on='business_id').filter('user_id = 100').show()

AnalysisException: USING column `business_id` cannot be resolved on the left side of the join. The left-side columns: [user_id_index, recommendations]

In [45]:
u_rec = nrecommendations.select("recommendations", "user_id_index").filter("user_id_index = 26").show()

+--------------------+-------------+
|     recommendations|user_id_index|
+--------------------+-------------+
|[{51, 4.9822407},...|           26|
+--------------------+-------------+



In [46]:
u_rec = nrecommendations.select("recommendations", "user_id_index").filter("user_id_index = 26")

In [49]:
u_rec.write.json('recom_output.json')

In [61]:
nrecommendations.printSchema()

root
 |-- user_id_index: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- business_id_index: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [60]:
nrecommendations.join(reviews_df, on='business_id_index').show()

AnalysisException: USING column `business_id_index` cannot be resolved on the left side of the join. The left-side columns: [user_id_index, recommendations]

In [59]:
reviews_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- business_id_index: double (nullable = false)
 |-- user_id: string (nullable = true)
 |-- user_id_index: double (nullable = false)
 |-- stars: double (nullable = true)



In [ ]:
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in ['business_id']]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(reviews_df).transform(reviews_df)
reviews_df = transformed.select(['business_id', 'business_id_index', 'user_id', 'user_id_index', 'stars'])
reviews_df.show()

In [62]:
reviews_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- business_id_index: double (nullable = false)
 |-- user_id: string (nullable = true)
 |-- user_id_index: double (nullable = false)
 |-- stars: double (nullable = true)



In [64]:
users_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- yelping_since: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)
 |-- cool: long (nullable = true)
 |-- fans: long (nullable = true)
 |-- average_stars: double (nullable = true)



In [67]:
users_df.show(4)

+--------------------+--------+------------+-------------------+------+-----+----+----+-------------+
|             user_id|    name|review_count|      yelping_since|useful|funny|cool|fans|average_stars|
+--------------------+--------+------------+-------------------+------+-----+----+----+-------------+
|hCAuMs7R7FFh4gbkM...|    Chon|         370|2009-12-25 23:15:20|   319|  127| 154|  14|         3.77|
|OyLO6fl4st6r_YX-L...|    Mish|          59|2009-06-23 22:52:52|    47|   18|  35|   2|          3.8|
|Qwc59G4wyGx7SvWsY...|Stephene|         137|2011-01-16 09:16:39|   182|   94| 104|   4|         3.82|
|Qwc59G4wyGx7SvWsY...|Stephene|         137|2011-01-16 09:16:39|   182|   94| 104|   4|         3.82|
+--------------------+--------+------------+-------------------+------+-----+----+----+-------------+
only showing top 4 rows



In [76]:
users_df.write.format('json').save("users_out3.json")

In [73]:
ujson = users_df.toJSON()

In [74]:
ujson

MapPartitionsRDD[19572] at toJavaRDD at NativeMethodAccessorImpl.java:0

In [82]:
reviews_df.write.format('json').save("users_out4.json")

In [89]:
busineses_sql_df = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:mysql://localhost/food_hunter_development")\
    .option("driver", "com.mysql.jdbc.Driver")\
    .option("dbtable", "restaurants").option("user", "root")\
    .option("password", "Nanu1996").load()

users_sql_df = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:mysql://localhost/food_hunter_development")\
    .option("driver", "com.mysql.jdbc.Driver")\
    .option("dbtable", "users").option("user", "root")\
    .option("password", "Nanu1996").load()

reviews_sql_df = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:mysql://localhost/food_hunter_development")\
    .option("driver", "com.mysql.jdbc.Driver")\
    .option("dbtable", "reviews").option("user", "root")\
    .option("password", "Nanu1996").load()

In [90]:
print(busineses_sql_df.columns)
print(users_sql_df.columns)
print(reviews_sql_df.columns)

['id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'categories', 'business_id', 'restaurant_image']
['id', 'email', 'encrypted_password', 'reset_password_token', 'reset_password_sent_at', 'remember_created_at', 'created_at', 'updated_at', 'reference_id']
['id', 'restaurant_id', 'user_id', 'stars', 'created_at', 'updated_at']


In [91]:
reviews_sql_df.show(5)

+-----+-------------+-------+-----+--------------------+--------------------+
|   id|restaurant_id|user_id|stars|          created_at|          updated_at|
+-----+-------------+-------+-----+--------------------+--------------------+
|16473|         2731|  11079|  1.0|2022-04-03 07:37:...|2022-04-03 07:37:...|
|16474|         2747|   8720|  1.0|2022-04-03 07:37:...|2022-04-03 07:37:...|
|16475|         2753|  11903|  5.0|2022-04-03 07:37:...|2022-04-03 07:37:...|
|16476|         2740|   4151|  1.0|2022-04-03 07:37:...|2022-04-03 07:37:...|
|16477|         2732|   3695|  3.0|2022-04-03 07:37:...|2022-04-03 07:37:...|
+-----+-------------+-------+-----+--------------------+--------------------+
only showing top 5 rows



In [88]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator


In [92]:
(train, test) = reviews_sql_df.randomSplit([0.8, 0.2], seed = 1234)
als = ALS(userCol="user_id", itemCol="restaurant_id", ratingCol="stars", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# Confirm that a model called "als" was created
type(als)


from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \

           
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))



cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)




#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel




# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())




# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)



nrecommendations = best_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

Num models to be tested:  16
CrossValidator_408668697c25


<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
  Rank: 150
  MaxIter: 10
  RegParam: 0.01
1.427511830619533


+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|   3566|[{2957, 4.999135}...|
|   3582|[{2861, 4.913816}...|
|   3587|[{2791, 4.999191}...|
|   3691|[{2855, 1.32571},...|
|   3706|[{2945, 1.0560126...|
|   3725|[{2962, 2.1782491...|
|   3761|[{2945, 2.2195144...|
|   3790|[{2846, 2.8143098...|
|   3794|[{2909, 3.9904087...|
|   3834|[{2791, 5.0344863...|
+-------+--------------------+



In [93]:
nrecommendations.show(20)

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|   3566|[{2957, 4.999135}...|
|   3582|[{2861, 4.913816}...|
|   3587|[{2791, 4.999191}...|
|   3691|[{2855, 1.32571},...|
|   3706|[{2945, 1.0560126...|
|   3725|[{2962, 2.1782491...|
|   3761|[{2945, 2.2195144...|
|   3790|[{2846, 2.8143098...|
|   3794|[{2909, 3.9904087...|
|   3834|[{2791, 5.0344863...|
|   3997|[{2827, 3.9992075...|
|   4042|[{2827, 4.999009}...|
|   4097|[{2861, 5.18926},...|
|   4119|[{2973, 4.2410398...|
|   4126|[{2945, 2.200633}...|
|   4132|[{2878, 4.4973173...|
|   4153|[{2945, 5.3666778...|
|   4167|[{2745, 3.1451046...|
|   4181|[{2945, 5.085301}...|
|   4186|[{2821, 4.9991784...|
+-------+--------------------+
only showing top 20 rows



In [94]:
nrecommendations.count()

9521

In [95]:
nrecommendations = best_model.recommendForAllUsers(20)

In [96]:
nrecommendations.limit(1).show()

+-------+--------------------+
|user_id|     recommendations|
+-------+--------------------+
|   3566|[{2957, 4.999135}...|
+-------+--------------------+



In [111]:
nrecommendations_json = nrecommendations.select('user_id', to_json(col('recommendations'))).withColumnRenamed("to_json(recommendations)","recommendations")

In [114]:
from pyspark.sql.functions import to_json,col
# # df2.withColumn("value",to_json(col("value"))) \
# #    .show(truncate=False)

nrecommendations_json.write.format('jdbc').options(
      url='jdbc:mysql://localhost/food_hunter_development',
      driver='com.mysql.jdbc.Driver',
      dbtable='user_recommendations',
      user='root',
      password='Nanu1996').mode('append').save()

In [125]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import to_json,col


reviews_sql_df = spark.read\
    .format("jdbc")\
    .option("url", "jdbc:mysql://localhost/food_hunter_development")\
    .option("driver", "com.mysql.jdbc.Driver")\
    .option("dbtable", "reviews").option("user", "root")\
    .option("password", "Nanu1996").load()


(train, test) = reviews_sql_df.randomSplit([0.8, 0.2], seed = 1234)
als = ALS(userCol="user_id", itemCol="restaurant_id", ratingCol="stars", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# Confirm that a model called "als" was created
type(als)



# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \

           
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))



cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)




#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel




# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())




# # View the predictions
# test_predictions = best_model.transform(test)
# RMSE = evaluator.evaluate(test_predictions)
# print(RMSE)


# nrecommendations = best_model.recommendForAllUsers(20)
# nrecommendations_json = nrecommendations.select('user_id', to_json(col('recommendations'))).withColumnRenamed("to_json(recommendations)","recommendations")

# nrecommendations_json.write.option("truncate", "true").format('jdbc').options(
#       url='jdbc:mysql://localhost/food_hunter_development',
#       driver='com.mysql.jdbc.Driver',
#       dbtable='user_recommendations',
#       user='root',
#       password='Nanu1996').mode("overwrite").save()


Num models to be tested:  16
CrossValidator_061c47aea2bb


<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
  Rank: 150
  MaxIter: 10
  RegParam: 0.01


In [126]:


# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)


nrecommendations = best_model.recommendForAllUsers(20)
nrecommendations_json = nrecommendations.select('user_id', to_json(col('recommendations'))).withColumnRenamed("to_json(recommendations)","recommendations")

nrecommendations_json.write.option("truncate", "true").format('jdbc').options(
      url='jdbc:mysql://localhost/food_hunter_development',
      driver='com.mysql.jdbc.Driver',
      dbtable='user_recommendations',
      user='root',
      password='Nanu1996').mode("overwrite").save()



1.4273658210501932
